# Multiple Linear Regression (MLR) Analysis

This notebook applies **Multiple Linear Regression (MLR)** to analyze the relationship between institutional characteristics and median earnings 10 years after enrollment. Unlike the Random Forest model, MLR provides interpretable coefficients, allowing us to quantify the influence of each feature on earnings in a linear fashion.

### Objective

The goal of this analysis is to:
1. Identify and quantify key factors that influence median earnings.
2. Interpret the direction and magnitude of each feature’s effect on the target variable.
3. Use MLR as a baseline for comparison with more complex models, such as the Random Forest, to assess trade-offs between interpretability and predictive accuracy.

### Approach

This analysis includes:
- Data preparation, including any transformations or scaling needed to optimize MLR performance.
- Model training and evaluation, focusing on interpreting the regression coefficients.
- Discussion of the implications of significant features, providing insights for institutions and policymakers.

This MLR model is a critical part of the overall project, as it offers an interpretable baseline model against which the performance of more complex models can be assessed.

---


## Data Preparation

To ensure consistency in our analysis, we replicate the data cleaning steps directly within this notebook. This includes handling missing values, creating new features, and performing any transformations necessary for optimal model performance.

The specific steps in this data preparation process are:
1. **Handling Missing Values**: Replacing or dropping missing values to ensure a complete dataset.
2. **Feature Engineering**: Creating new features, such as combining related variables or creating binary indicators, to enhance the model’s predictive power.
3. **Data Transformation**: Scaling or encoding variables as needed to align with the requirements of Multiple Linear Regression.

By replicating the data cleaning process, we maintain independence between this notebook and other models, ensuring reproducibility and a consistent methodology.


In [ ]:
# Imports
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm

In [ ]:
# Importing dataset
df = pd.read_csv('C:/Users/ajten/Downloads/College_Scorecard_Raw_Data_10022024/College_Scorecard_Raw_Data_09242024/MERGED2014_15_PP.csv')
df.info()
df.describe()

# Variables to keep 
variables = [
    'CONTROL', 'UGDS','AVGFACSAL', 'PCTPELL', 'COSTT4_A', 'COSTT4_P', 'MD_FAMINC',
    'MD_EARN_WNE_P10', 'INEXPFTE', 'C150_4', 'C150_L4', 'PCTFLOAN', 'DEBT_MDN', 
    'PCIP14', 'PCIP11', 'PCIP15', 'PCIP52', 'STUFACR'
]

# Subset the dataframe
df = df[variables]

# Display the shape of the dataset
print("Dataset shape:", df.shape)

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
for col in df.columns:
    if 'PS' in df[col].values:
        print(f"Column '{col}' contains 'PS'")

In [ ]:
# List of columns that might contain 'PrivacySuppressed'
columns_to_clean = ['MD_EARN_WNE_P10', 'MD_FAMINC', 'DEBT_MDN']

# Replace 'PrivacySuppressed (PS)' with NaN
df[columns_to_clean] = df[columns_to_clean].replace('PS', np.nan)

# Convert the columns to numeric
for col in columns_to_clean:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Drop rows where any NaN values exist in the specified columns to clean
df_clean = df.dropna(subset=columns_to_clean).copy()

In [ ]:
df_clean.isnull().sum()

In [ ]:
# Create a general completion rate variable that combines C150_4 and C150_L4
df_clean['completion_rate'] = df_clean['C150_4'].combine_first(df_clean['C150_L4'])

# Create a general completion rate variable that combines C150_4 and C150_L4
df_clean['avg_attendance'] = df_clean['COSTT4_A'].combine_first(df_clean['COSTT4_P'])

# Handle missing values in critical variables and make a copy
critical_vars = ['UGDS','AVGFACSAL','PCTPELL','INEXPFTE', 'completion_rate', 'avg_attendance', 'DEBT_MDN']
df_clean = df_clean.dropna(subset=critical_vars).copy()

# Create binary variable for institution type
# 0 = Public, 1 = Private (combining nonprofit and for-profit)
df_clean['public_private'] = df_clean['CONTROL'].apply(lambda x: 0 if x == 1 else 1)

# Dropping redundant variables
df3 = df_clean.drop(['CONTROL','C150_4', 'C150_L4', 'COSTT4_A', 'COSTT4_P'], axis=1)

df3.isnull().sum()

In [ ]:
# Final dataset
print("Cleaned Dataset Shape:", df3.shape)
df3.info()

## Exploratory Data Analysis (EDA)

For this analysis, we performed a streamlined EDA focusing on descriptive statistics and correlation analysis to identify potential relationships between institutional characteristics and median earnings 10 years after enrollment.

### Descriptive Statistics

We began with descriptive statistics to understand the distribution, central tendency, and variability of each variable. This provides an overview of the data, allowing us to identify any features with high variance or unusual distributions that may influence the model’s performance.

### Correlation Matrix

The correlation matrix heatmap displays the pairwise correlations between all variables in the dataset. This analysis helps identify features with strong linear relationships with the target variable (`MD_EARN_WNE_P10`) and among themselves, providing insights into which variables might be more influential in predicting median earnings.

#### Key Observations:
- **Strong Positive Correlations**: `MD_FAMINC` (Family Income) and `AVGFACSAL` (Average Faculty Salary) have high positive correlations with median earnings, suggesting these socioeconomic and institutional factors are closely tied to graduate outcomes.
- **Strong Negative Correlation**: `PCTPELL` (Percentage of Pell Grant Recipients) shows a notable negative correlation with median earnings, indicating that institutions with higher proportions of Pell Grant recipients generally have lower post-graduation earnings among their graduates.
- **Moderate Correlations**: Features like `avg_attendance` (Average Cost of Attendance), `DEBT_MDN` (Median Debt), and `STEM_Index` show moderate positive correlations with median earnings, which might indicate their relevance to the model.

This initial analysis highlights the features that could have the most significant impact on median earnings, setting the stage for the subsequent regression analysis.


In [ ]:
# Descriptive statistics
print("Descriptive Statistics:")
print(df3[['public_private', 'UGDS','AVGFACSAL', 'PCTPELL', 'avg_attendance', 'MD_FAMINC',
    'MD_EARN_WNE_P10', 'INEXPFTE', 'completion_rate', 'PCTFLOAN', 'DEBT_MDN', 
    'PCIP14', 'PCIP11', 'PCIP15', 'PCIP52', 'STUFACR']].describe())

In [ ]:
# Correlation matrix
corr_vars = ['public_private', 'UGDS','AVGFACSAL', 'PCTPELL', 'avg_attendance', 'MD_FAMINC',
    'MD_EARN_WNE_P10', 'INEXPFTE', 'completion_rate', 'PCTFLOAN', 'DEBT_MDN', 
    'PCIP14', 'PCIP11', 'PCIP15', 'PCIP52', 'STUFACR']
corr_matrix = df3[corr_vars].corr()

print("Correlation Matrix:")
print(corr_matrix)

# Heatmap of the correlation matrix
plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap='mako')
plt.title('Correlation Matrix Heatmap')
plt.show()

## Discretization and Transformations

To address skewness and scale differences in the data, we applied several transformations to specific variables. These transformations help ensure that the features align more closely with the assumptions of linear regression, improving model performance and interpretability.

### Log Transformations
Due to right-skewed distributions, we applied a log transformation to certain continuous variables. The log transformation helps reduce skewness, making the data more symmetric and suitable for linear modeling. The following features were log-transformed:
- **UGDS**: Undergraduate enrollment.
- **AVGFACSAL**: Average faculty salary.
- **MD_FAMINC**: Median family income.
- **INEXPFTE**: Instructional expenditure per full-time equivalent student.
- **avg_attendance**: Average cost of attendance.
- **DEBT_MDN**: Median debt of graduates.
- **MD_EARN_WNE_P10**: Median earnings 10 years after enrollment (target variable).

### Data Filtering
After the log transformation, we observed that certain rows in `MD_FAMINC_log` (transformed family income) still exhibited skew. To address this, we filtered out rows where `MD_FAMINC_log` was below 0.6, improving the distribution of the data and reducing the impact of extreme outliers.

### Distribution Visualization
Finally, we plotted histograms for key features to verify the effectiveness of the transformations. The improved distributions confirm that the transformations were successful in reducing skew and preparing the data for modeling.



In [ ]:
df3['UGDS_log'] = np.log1p(df3['UGDS'])
df3['AVGFACSAL_log'] = np.log1p(df3['AVGFACSAL'])
df3['MD_FAMINC_log'] = np.log1p(df3['MD_FAMINC'])
df3['INEXPFTE_log'] = np.log1p(df3['INEXPFTE'])
df3['avg_attendance_log'] = np.log1p(df3['avg_attendance'])
df3['DEBT_MDN_log'] = np.log1p(df3['DEBT_MDN'])
df3['MD_EARN_WNE_P10_log'] = np.log1p(df3['MD_EARN_WNE_P10'])

In [ ]:
# Remove rows where MD_FAMINC_log is below 0.6
df3 = df3[df3['MD_FAMINC_log'] >= 0.6]

In [ ]:
# Sum of proportions of degrees in Engineering, Computer Science, and other STEM fields
df3['STEM_Index'] = df3['PCIP14'] + df3['PCIP11'] + df3['PCIP15'] + df3['PCIP52']

# Plot histograms to check distribution
df3[['public_private', 'UGDS_log','AVGFACSAL_log', 'PCTPELL', 'avg_attendance_log', 'MD_FAMINC_log',
    'INEXPFTE_log', 'completion_rate', 'PCTFLOAN', 'DEBT_MDN_log', 'STEM_Index',
    'STUFACR', 'MD_EARN_WNE_P10_log']].hist(bins=30, figsize=(10, 8),edgecolor='black')
plt.subplots_adjust(hspace=0.5, wspace=0.4)
plt.show()

## Multicollinearity Assessment and Feature Selection/Reduction

To ensure that our model is stable and interpretable, we assessed multicollinearity among the features using the **Variance Inflation Factor (VIF)**. High multicollinearity can inflate the standard errors of coefficients, making it difficult to determine the independent effect of each feature on the target variable.

### Variance Inflation Factor (VIF)

The VIF for each feature was calculated as follows:
- **VIF < 5**: Indicates low multicollinearity, suggesting the feature can be reliably included in the model.
- **VIF >= 5**: Indicates high multicollinearity, suggesting the feature may be redundant and could distort model interpretation.

#### Key Findings:
- Most features have VIF values below 5, indicating acceptable levels of multicollinearity for this analysis.
- The `DEBT_MDN_log` feature shows a slightly higher VIF (around 3.73), but it remains below the commonly used threshold of 5, so it was retained.
- The constant term (added for model stability) has a high VIF, but this is typical and does not affect the interpretation of other features.

This assessment confirms that the features included in the model do not exhibit problematic multicollinearity, allowing us to proceed with a stable and interpretable Multiple Linear Regression model.


In [ ]:
# Recalculating VIF
X = df3[['UGDS_log','AVGFACSAL_log', 'PCTPELL', 'avg_attendance_log', 'MD_FAMINC_log',
    'completion_rate','DEBT_MDN_log', 'STEM_Index', 'INEXPFTE_log']]

X = X.assign(const=1)

# Calculate VIF for each feature
vif_data = pd.DataFrame()
vif_data['Feature'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(len(X.columns))]

print("Variance Inflation Factors:")
print(vif_data)

## Multiple Linear Regression (MLR) Analysis

To quantify the relationship between institutional characteristics and median earnings 10 years after enrollment, we applied **Ordinary Least Squares (OLS) Regression**. This model provides interpretable coefficients that allow us to understand the impact of each feature on the target variable (`MD_EARN_WNE_P10_log`).

In [ ]:
# Define X with only the main effect variables
X = df3[['UGDS_log','AVGFACSAL_log', 'PCTPELL', 'avg_attendance_log', 'MD_FAMINC_log',
    'completion_rate','DEBT_MDN_log', 'STEM_Index', 'INEXPFTE_log']]
X = sm.add_constant(X)  # Adds an intercept to the model

# Define the target variable (log-transformed)
y = df3['MD_EARN_WNE_P10_log']

# Fit the MLR model
mlr_model = sm.OLS(y, X).fit()

# Display the summary of the model results
print(mlr_model.summary())


## Model Summary
- **R-squared**: 0.703  
  This indicates that approximately 70.3% of the variance in median earnings (log-transformed) is explained by the features in the model.
- **Adjusted R-squared**: 0.702  
  Similar to the R-squared, this adjusted measure accounts for the number of predictors in the model and suggests a strong fit.
- **F-statistic**: 937.3 (p < 0.001)  
  A highly significant F-statistic indicates that the model as a whole is statistically significant.

### Coefficients and Interpretation
Below are the key features with their coefficients, t-statistics, and p-values. Significant coefficients (p < 0.05) indicate a statistically meaningful relationship with the target variable.

- **Constant (Intercept)**: 6.218  
  Represents the baseline log-transformed earnings when all predictor variables are zero.

- **UGDS_log** (Undergraduate enrollment): 0.028 (p < 0.001)  
  A positive coefficient suggests that higher undergraduate enrollment is associated with slightly higher median earnings.

- **AVGFACSAL_log** (Average faculty salary): 0.132 (p < 0.001)  
  Higher faculty salaries are positively associated with graduate earnings, likely reflecting institutional quality.

- **PCTPELL** (Percentage of Pell Grant recipients): -0.288 (p < 0.001)  
  This negative coefficient indicates that institutions with higher proportions of Pell Grant recipients tend to have lower median earnings, potentially reflecting socioeconomic disparities.

- **avg_attendance_log** (Average cost of attendance): 0.088 (p < 0.001)  
  Higher attendance costs are associated with higher earnings, possibly indicating that more expensive institutions offer higher returns.

- **MD_FAMINC_log** (Median family income): 0.111 (p < 0.001)  
  Family income is positively correlated with earnings, suggesting that socioeconomic background influences long-term outcomes.

- **completion_rate**: 0.043 (p = 0.009)  
  Higher completion rates are modestly associated with higher earnings, underscoring the value of student success and retention programs.

- **DEBT_MDN_log** (Median debt): 0.077 (p < 0.001)  
  Although debt has a positive association with earnings, it likely represents investment in education that correlates with improved outcomes.

- **INEXPFTE_log** (Instructional expenditure per FTE): 0.024 (p < 0.001)  
  Higher instructional spending per student correlates with improved earnings, potentially reflecting institutional resources and educational quality.

### Model Diagnostics
- **Durbin-Watson Statistic**: 1.682  
  Indicates no serious autocorrelation in residuals, suggesting that the error terms are independent.
- **Omnibus and Jarque-Bera Tests**: Significant values indicate slight skewness and kurtosis in residuals, which may affect normality assumptions but are within tolerable limits.

## Conclusion
This Multiple Linear Regression model highlights key institutional and socioeconomic factors associated with earnings outcomes. The positive relationships with faculty salary, family income, and instructional spending underscore the importance of resources in influencing graduate earnings. Conversely, the negative association with Pell Grant percentages suggests areas where institutions can work to improve outcomes for low-income students.
